## Insurance Staff Planning Optimisation Case Study
### Abhijeet Srivastava DS C32 BA
#### Assigned partner never replied to my emails/calls

In [ ]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math


#installing pyomo for Colab
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

from __future__ import division
from pyomo.opt import SolverFactory

In [ ]:
#mounting Google Drive to access dataset 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab_Notebooks/Case_Study_OR/'


In [ ]:
# Reading the data from Excel workbook
#InputData = base_dir + "Staffing+Data.xlsx"

#reading the data by manually uploading to Google Colab as Colab can't find the dataset in the directory
InputData = "Staffing+Data.xlsx"

StaffAv = pd.read_excel(InputData, sheet_name="StaffAvailability")

Cost = pd.read_excel(InputData, sheet_name="Cost")

transcost = pd.read_excel(InputData, sheet_name="ServiceRate")

MgDemandDist = pd.read_excel(InputData, sheet_name="DemandData")


In [ ]:
StaffAv.head()

In [ ]:
Cost.head()

In [ ]:
transcost.head()

In [ ]:
MgDemandDist.head()

## Data pre-processing 

In [ ]:
# Create the required Python data structures for indexes and parameters
# 1 Demand Data
Demand = MgDemandDist.set_index(["State","Month"])["Demand"].to_dict()

# 2 FTE Salary from the data
FTE_Salary = Cost.set_index(["State","Month"])["MonthlySalary"].to_dict()

# 3 Unit OutSourcing Cost
UnitOutSourceCost = Cost.set_index(["State","Month"])['UnitOutSourceCost'].to_dict()

# 4 Number of Applications when employee working 100%
FTE_AppServeRate = transcost.iloc[0,0]

# 5 Staff Availability for serving the Insurance Applications
StaffAv_LB = StaffAv.set_index(["State","Month"])["LB"].to_dict()
StaffAv_UB = StaffAv.set_index(["State","Month"])["UB"].to_dict()
StaffAvPer = StaffAv.set_index(["State","Month"])["StaffAvPer"].to_dict()

# 6 Index
BankLoc = Cost['State'].unique()
Month = Cost["Month"].unique()

In [ ]:
BankLoc 

array(['A', 'B', 'C'], dtype=object)

In [ ]:
Month

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [ ]:
BankLoc.tolist()

['A', 'B', 'C']

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



###  Mathematical formulation / Pyomo components

---

**Index Definition:** 

The indexes for the given problem are, <br>
- i = Bank Location, 
- j = Month

---

**Parameters:** 

- $\text{Demand}_{i,j} \text{ - Demand for the bank location,i, for the given month,j}$ <br>
- $\text{StaffAv}_{i,j} \text{ - Staff Availability percentage for location i and month j}$ <br>
- $\text{FTE_Salary}_{i,j} \text{ - Full Time Employee Salary per month}$ <br>
- $\text{UnitCost}_{i,j} \text{ - Unit cost for outsourcing for location i and month j}$ <br>
- $\text{FTE_AppServeRate}: 40 \text{ - Max number of applications per month working at 100% i.e. FTE=1}$ <br>

---

**Decision variables:**
- FTE Equivalent at any location in a month (Continuous varaible) <br>

> ${X}_{i,j} \,\,\,\,\,\,\, \text{where} \ i \in location, j \in month $<br>

- Outsourced Insurance Application Quantity in a month (Integer variable)

>${Y}_{i,j} \,\,\,\,\,\,\, \text{where} \ i \in location, j \in month $<br>
---

**Objective Function:** <br>

1) Annual total cost of FTE

2) Annual total cost of Outsourced Insurance Applications 

To <b>Minimise</b> total cost = 1) + 2) 


\begin{align}
\textrm{min} \sum \limits _{i} \sum \limits _{j} \text{X}_{i,j}* \text{FTE_Salary}_{i,j} \ + \sum \limits _{i} \sum \limits _{j} \text{Y}_{i,j}* \text{UnitCost}_{i,j}
\end{align}

where $i \in Location $ and $j \in Month$

---

**Constraints:**



*   (FTE_Equivalent/Month)*(40 Applications/FTE)*Availability + (Outsourced Insurance Application Quantity) has to be equal to the Demand per month, i.e.


>- $\text{X}_{i,j} \text{*40*} \text{StaffAv}_{i,j} \text{+Y}_{i,j} \ = \text{Demand}_{i,j}\ \ \ \ \forall {i \in Location, j \in Month}$


*  Locations A & B have regulatory constraints on  the number of outsourced application


>- $\text{Y}_{i,j} \leq \text{Demand}_{i,j} \text{*0.3} \forall {i \in A, j \in Month}$
>- $\text{Y}_{i,j} \leq \text{Demand}_{i,j} \text{*0.4} \forall {i \in B, j \in Month}$


*   Outsourced Insurance Applications have to be intergers
>- $\text{Y}_{i,j}\text{ in Integers} $


*   Full Time Employee (FTE) Equivalent at any location in a month has to be a Continuous variable
>- $\text{X}_{i,j} \text{ in Continuous} $


----



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [ ]:
# Creating a model instance
model = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
# Define Sets
# i = States Loc, j = Months

model.i = Set(initialize=BankLoc.tolist(), doc="States")
model.j = Set(initialize=Month.tolist(), doc="Months")

# Define parameters for Demand, FTE Salaries, OutSourcing Cost and Average Staff Availability
model.d = Param(model.i, model.j, initialize=Demand, doc="Demand")
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc="FTE_Salary")
model.oc = Param(model.i, model.j, initialize=UnitOutSourceCost, doc="OutSourceCost")
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc="StaffAvPercent")

# Appl Service Rate when working 100% -> Scalar
model.r = Param(initialize=FTE_AppServeRate, doc="FTE App Serve Rate")

In [ ]:
# Parameter for no of appl that can be processed for the given staff avail

def c_FTEAPP(model, i, j):
    return model.r*model.sa[i,j]

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc="NO OF FTE APP")

In [ ]:
# Parameter for restriction Location A 

def Reg_A(model, i, j):
    return 0.30 * model.d[i,j]

model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc="RegRest_A")

In [ ]:
# Parameter for restriction Location B

def Reg_B(model, i, j):
    return 0.40 * model.d[i,j]

model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc="RegRest_B")

In [ ]:
# Decision variables
model.x = Var(model.i, model.j, domain=NonNegativeReals, doc="No of FTE")

model.y = Var(model.i, model.j, domain=NonNegativeIntegers, doc="No of Outsourced App")

In [ ]:
#Demand Constraints
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr= model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [ ]:
# Regulatory Constraint

model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i=="A":
            model.regulatoryCons.add(expr= model.y['A',j] <= model.Reg_A['A',j])
        elif i=="B":
            model.regulatoryCons.add(expr= model.y['B',j] <= model.Reg_B['B',j])

In [ ]:
# Objective function
def objective_rule(model):
    return sum(model.x[i,j]*model.s[i,j] for i in model.i for j in model.j) +\
           sum(model.y[i,j]*model.oc[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective function")

In [ ]:
# Invoking the solver
result_cost = SolverFactory("glpk").solve(model)

result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015081644058227539
# ----------------------------------------------------------
#   Solution Information
# -----------

In [ ]:
# Print the value of the objective function
Total_Cost = model.objective.expr()
Total_Cost

17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
# Creating dataframe for the results
FTE_Staff = []
for i in BankLoc:
    for j in Month:
        v = model.x[i,j].value
        w = model.y[i,j].value
        FTE_Staff.append([i,j,v,w])
        
FTE_Staff

[['A', 'Jan', 161.728395061728, 0.0],
 ['A', 'Feb', 160.460526315789, 0.0],
 ['A', 'Mar', 198.066666666667, 0.0],
 ['A', 'Apr', 71.78125, 0.0],
 ['A', 'May', 63.8461538461538, 0.0],
 ['A', 'Jun', 77.9109589041096, 0.0],
 ['A', 'Jul', 137.279411764706, 1600.0],
 ['A', 'Aug', 110.888157894737, 0.0],
 ['A', 'Sep', 116.018518518519, 0.0],
 ['A', 'Oct', 120.856164383562, 0.0],
 ['A', 'Nov', 110.257352941176, 1285.0],
 ['A', 'Dec', 139.576923076923, 1554.0],
 ['B', 'Jan', 152.067901234568, 0.0],
 ['B', 'Feb', 51.875, 1051.0],
 ['B', 'Mar', 59.5, 1189.0],
 ['B', 'Apr', 73.0625, 0.0],
 ['B', 'May', 128.846153846154, 0.0],
 ['B', 'Jun', 64.6917808219178, 1258.0],
 ['B', 'Jul', 94.2279411764706, 1708.0],
 ['B', 'Aug', 51.7105263157895, 1048.0],
 ['B', 'Sep', 139.413580246914, 0.0],
 ['B', 'Oct', 85.3767123287671, 1662.0],
 ['B', 'Nov', 69.2279411764706, 1254.0],
 ['B', 'Dec', 97.5769230769231, 1690.0],
 ['C', 'Jan', 35.8641975308642, 0.0],
 ['C', 'Feb', 64.7039473684211, 0.0],
 ['C', 'Mar', 63.2

In [ ]:
FTE_Staff = pd.DataFrame(FTE_Staff, columns=['State',"Month","FTE","Outsourced"])
FTE_Staff

,State,Month,FTE,Outsourced
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0
5,A,Jun,77.910959,0.0
6,A,Jul,137.279412,1600.0
7,A,Aug,110.888158,0.0
8,A,Sep,116.018519,0.0
9,A,Oct,120.856164,0.0


In [ ]:
# Writing the results in to an Excel sheet
OutputData =  base_dir + "Output+Staffing+Data.xlsx"
FTE_Staff.to_excel(OutputData, sheet_name="Q2_Output", index='False')

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model_lb = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
# i = States Loc, j = Months

model_lb.i = Set(initialize=BankLoc.tolist(), doc="States")
model_lb.j = Set(initialize=Month.tolist(), doc="Months")

# Define parameters for Demand, FTE Salaries, OutSourcing Cost and Average Staff Availability
model_lb.d = Param(model.i, model.j, initialize=Demand, doc="Demand")
model_lb.s = Param(model.i, model.j, initialize=FTE_Salary, doc="FTE_Salary")
model_lb.oc = Param(model.i, model.j, initialize=UnitOutSourceCost, doc="OutSourceCost")
model_lb.lb = Param(model.i, model.j, initialize=StaffAv_LB, doc="StaffAv_LB")

# Appl Service Rate when working 100% -> Scalar
model_lb.r = Param(initialize=FTE_AppServeRate, doc="FTE App Serve Rate")

In [ ]:
# Parameter for no of appl that can be processed for the given staff avail

def c_FTEAPP(model_lb, i, j):
    return model_lb.r*model_lb.lb[i,j]

model_lb.FTEAPPNO = Param(model_lb.i, model_lb.j, initialize=c_FTEAPP, doc="NO OF FTE APP")

In [ ]:
# Parameter for restriction A

def Reg_A(model_lb, i, j):
    return 0.30 * model_lb.d[i,j]

model_lb.Reg_A = Param(model_lb.i, model_lb.j, initialize=Reg_A, doc="RegRest_A")

# Parameter for restriction B

def Reg_B(model_lb, i, j):
    return 0.40 * model_lb.d[i,j]

model_lb.Reg_B = Param(model_lb.i, model_lb.j, initialize=Reg_B, doc="RegRest_B")


In [ ]:
# Define Decision Variables

model_lb.x = Var(model_lb.i, model_lb.j, domain=NonNegativeReals, doc="No of FTE")

model_lb.y = Var(model_lb.i, model_lb.j, domain=NonNegativeIntegers, doc="No of Outsourced App")


In [ ]:
# Demand Constraint

model_lb.demand_constraint = ConstraintList()
for i in model_lb.i:
    for j in model_lb.j:
        model_lb.demand_constraint.add(expr= model_lb.x[i,j]*model_lb.FTEAPPNO[i,j] + model_lb.y[i,j] == model_lb.d[i,j])


In [ ]:
# Regulatory Constraint

model_lb.regulatoryCons = ConstraintList()
for i in model_lb.i:
    for j in model_lb.j:
        if i=="A":
            model_lb.regulatoryCons.add(expr= model_lb.y['A',j] <= model_lb.Reg_A['A',j])
        elif i=="B":
            model_lb.regulatoryCons.add(expr= model_lb.y['B',j] <= model_lb.Reg_B['B',j])

In [ ]:
# Objective function

def objective_rule(model_lb):
    return sum(model_lb.x[i,j]*model_lb.s[i,j] for i in model_lb.i for j in model_lb.j) +\
           sum(model_lb.y[i,j]*model_lb.oc[i,j] for i in model_lb.i for j in model_lb.j)

model_lb.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective function")

In [ ]:
# Invoking the solver
result_cost_lb = SolverFactory("glpk").solve(model_lb)

result_cost_lb.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01682758331298828
# ----------------------------------------------------------
#   Solution Information
# ------------

In [ ]:
# Print the value of the objective function
Total_Cost_lb = model_lb.objective.expr()
Total_Cost_lb

19599482.516788766

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
# Creating dataframe for the results
FTE_Staff_lb = []
for i in BankLoc:
    for j in Month:
        v = model_lb.x[i,j].value
        w = model_lb.y[i,j].value
        FTE_Staff_lb.append([i,j,v,w])
        
FTE_Staff_lb

[['A', 'Jan', 187.142857142857, 0.0],
 ['A', 'Feb', 131.346153846154, 1463.0],
 ['A', 'Mar', 212.214285714286, 0.0],
 ['A', 'Apr', 76.5666666666667, 0.0],
 ['A', 'May', 71.1428571428571, 0.0],
 ['A', 'Jun', 61.2692307692308, 682.0],
 ['A', 'Jul', 155.583333333333, 1600.0],
 ['A', 'Aug', 90.7692307692308, 1011.0],
 ['A', 'Sep', 134.25, 0.0],
 ['A', 'Oct', 95.0384615384615, 1058.0],
 ['A', 'Nov', 124.958333333333, 1285.0],
 ['A', 'Dec', 151.208333333333, 1554.0],
 ['B', 'Jan', 105.607142857143, 1970.0],
 ['B', 'Feb', 60.6538461538462, 1051.0],
 ['B', 'Mar', 63.75, 1189.0],
 ['B', 'Apr', 46.7666666666667, 935.0],
 ['B', 'May', 86.1428571428571, 1608.0],
 ['B', 'Jun', 72.6538461538462, 1258.0],
 ['B', 'Jul', 106.791666666667, 1708.0],
 ['B', 'Aug', 60.4615384615385, 1048.0],
 ['B', 'Sep', 96.8214285714286, 1806.0],
 ['B', 'Oct', 95.8846153846154, 1662.0],
 ['B', 'Nov', 78.4583333333333, 1254.0],
 ['B', 'Dec', 105.708333333333, 1690.0],
 ['C', 'Jan', 41.5, 0.0],
 ['C', 'Feb', 0.0, 1967.0],


In [ ]:
FTE_Staff_lb = pd.DataFrame(FTE_Staff_lb, columns=['State',"Month","FTE","Outsourced"])
FTE_Staff_lb

,State,Month,FTE,Outsourced
0,A,Jan,187.142857,0.0
1,A,Feb,131.346154,1463.0
2,A,Mar,212.214286,0.0
3,A,Apr,76.566667,0.0
4,A,May,71.142857,0.0
5,A,Jun,61.269231,682.0
6,A,Jul,155.583333,1600.0
7,A,Aug,90.769231,1011.0
8,A,Sep,134.250000,0.0
9,A,Oct,95.038462,1058.0


In [ ]:
# Writing the results in to an Excel sheet
OutputData_3_1 =  base_dir + "Output+Staffing+Data_3_1.xlsx"
FTE_Staff_lb.to_excel(OutputData_3_1, sheet_name="Q3_Output_1", index='False')

#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here
#number of outsourced apps -> 1)
Outsourced_apps_lb = FTE_Staff_lb.Outsourced.sum()
#total apps -> 2)
Total_Apps = MgDemandDist.Demand.sum()
#Percentage = 1)/2)
Perc_outsourced_lb = round(Outsourced_apps_lb/Total_Apps,2)
Perc_outsourced_lb

0.35

#### 3.1.3 Average cost per application

In [ ]:
# write your code here
Total_Apps = MgDemandDist.Demand.sum()
Avg_cost_lb = Total_Cost_lb/Total_Apps
Avg_cost_lb

172.9966504562357

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model_ub = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
# i = States Loc, j = Months

model_ub.i = Set(initialize=BankLoc.tolist(), doc="States")
model_ub.j = Set(initialize=Month.tolist(), doc="Months")

# Define parameters for Demand, FTE Salaries, OutSourcing Cost and Average Staff Availability
model_ub.d = Param(model.i, model.j, initialize=Demand, doc="Demand")
model_ub.s = Param(model.i, model.j, initialize=FTE_Salary, doc="FTE_Salary")
model_ub.oc = Param(model.i, model.j, initialize=UnitOutSourceCost, doc="OutSourceCost")
model_ub.ub = Param(model.i, model.j, initialize=StaffAv_UB, doc="StaffAv_UB")

# Appl Service Rate when working 100% -> Scalar
model_ub.r = Param(initialize=FTE_AppServeRate, doc="FTE App Serve Rate")

In [ ]:
# Parameter for no of appl that can be processed for the given staff avail

def c_FTEAPP(model_ub, i, j):
    return model_ub.r*model_ub.ub[i,j]

model_ub.FTEAPPNO = Param(model_ub.i, model_ub.j, initialize=c_FTEAPP, doc="NO OF FTE APP")

In [ ]:
# Parameter for restriction A

def Reg_A(model_ub, i, j):
    return 0.30 * model_ub.d[i,j]

model_ub.Reg_A = Param(model_ub.i, model_ub.j, initialize=Reg_A, doc="RegRest_A")

# Parameter for restriction B

def Reg_B(model_ub, i, j):
    return 0.40 * model_ub.d[i,j]

model_ub.Reg_B = Param(model_ub.i, model_ub.j, initialize=Reg_B, doc="RegRest_B")


In [ ]:
# Define Decision Variables

model_ub.x = Var(model_ub.i, model_ub.j, domain=NonNegativeReals, doc="No of FTE")

model_ub.y = Var(model_ub.i, model_ub.j, domain=NonNegativeIntegers, doc="No of Outsourced App")


In [ ]:
# Demand Constraint

model_ub.demand_constraint = ConstraintList()
for i in model_ub.i:
    for j in model_ub.j:
        model_ub.demand_constraint.add(expr= model_ub.x[i,j]*model_ub.FTEAPPNO[i,j] + model_ub.y[i,j] == model_ub.d[i,j])


In [ ]:
# Regulatory Constraint

model_ub.regulatoryCons = ConstraintList()
for i in model_ub.i:
    for j in model_ub.j:
        if i=="A":
            model_ub.regulatoryCons.add(expr= model_ub.y['A',j] <= model_ub.Reg_A['A',j])
        elif i=="B":
            model_ub.regulatoryCons.add(expr= model_ub.y['B',j] <= model_ub.Reg_B['B',j])

In [ ]:
# Objective function
def objective_rule(model_ub):
    return sum(model_ub.x[i,j]*model_ub.s[i,j] for i in model_ub.i for j in model_ub.j) +\
           sum(model_ub.y[i,j]*model_ub.oc[i,j] for i in model_ub.i for j in model_ub.j)

model_ub.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective function")

In [ ]:
# Invoking the solver
result_cost_ub = SolverFactory("glpk").solve(model_ub)

result_cost_ub.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012673377990722656
# ----------------------------------------------------------
#   Solution Information
# -----------

In [ ]:
# Print the value of the objective function
Total_Cost_ub = model_ub.objective.expr()
Total_Cost_ub

16527535.63793573

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
# Creating dataframe for the results
FTE_Staff_ub = []
for i in BankLoc:
    for j in Month:
        v = model_ub.x[i,j].value
        w = model_ub.y[i,j].value
        FTE_Staff_ub.append([i,j,v,w])
        
FTE_Staff_ub

[['A', 'Jan', 145.555555555556, 0.0],
 ['A', 'Feb', 143.470588235294, 0.0],
 ['A', 'Mar', 185.6875, 0.0],
 ['A', 'Apr', 67.5588235294118, 0.0],
 ['A', 'May', 58.5882352941176, 0.0],
 ['A', 'Jun', 71.09375, 0.0],
 ['A', 'Jul', 177.8, 0.0],
 ['A', 'Aug', 99.1470588235294, 0.0],
 ['A', 'Sep', 104.416666666667, 0.0],
 ['A', 'Oct', 110.28125, 0.0],
 ['A', 'Nov', 142.8, 0.0],
 ['A', 'Dec', 185.107142857143, 0.0],
 ['B', 'Jan', 136.861111111111, 0.0],
 ['B', 'Feb', 77.2941176470588, 0.0],
 ['B', 'Mar', 92.9375, 0.0],
 ['B', 'Apr', 68.7647058823529, 0.0],
 ['B', 'May', 118.235294117647, 0.0],
 ['B', 'Jun', 98.34375, 0.0],
 ['B', 'Jul', 85.4333333333333, 1708.0],
 ['B', 'Aug', 77.0588235294118, 0.0],
 ['B', 'Sep', 125.472222222222, 0.0],
 ['B', 'Oct', 129.84375, 0.0],
 ['B', 'Nov', 62.7666666666667, 1254.0],
 ['B', 'Dec', 90.6071428571429, 1690.0],
 ['C', 'Jan', 32.2777777777778, 0.0],
 ['C', 'Feb', 57.8529411764706, 0.0],
 ['C', 'Mar', 59.3125, 0.0],
 ['C', 'Apr', 66.5, 0.0],
 ['C', 'May', 59.

In [ ]:
# Creating dataframe for the results

FTE_Staff_ub = pd.DataFrame(FTE_Staff_ub, columns=['State',"Month","FTE","Outsourced"])
FTE_Staff_ub

,State,Month,FTE,Outsourced
0,A,Jan,145.555556,0.0
1,A,Feb,143.470588,0.0
2,A,Mar,185.687500,0.0
3,A,Apr,67.558824,0.0
4,A,May,58.588235,0.0
5,A,Jun,71.093750,0.0
6,A,Jul,177.800000,0.0
7,A,Aug,99.147059,0.0
8,A,Sep,104.416667,0.0
9,A,Oct,110.281250,0.0


In [ ]:
# Writing the results in to an Excel sheet
OutputData_3_2 =  base_dir + "Output+Staffing+Data_3_2.xlsx"
FTE_Staff_ub.to_excel(OutputData_3_2, sheet_name="Q3_Output_2", index='False')

#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here
#Similarly as 3.1.2
Outsourced_apps_ub = FTE_Staff_ub.Outsourced.sum()
Total_Apps = MgDemandDist.Demand.sum()
Perc_outsourced_ub = round(Outsourced_apps_ub/Total_Apps,2)
Perc_outsourced_ub

0.04

#### 3.2.3 Average cost per application

In [ ]:
# write your code here
Total_Apps = MgDemandDist.Demand.sum()
Avg_cost = Total_Cost_ub/Total_Apps
Avg_cost

145.88182638035315

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.